In [3]:
from dotenv import load_dotenv
load_dotenv()

True

# 1. Indexing Phase

### (1) Load

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('./data/snow-white.pdf')
documents = loader.load()

In [6]:
# 개행문자 제거
for doc in documents:
    doc.page_content = doc.page_content.replace('\n', ' ')

documents

[Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': './data/snow-white.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='백설공주 옛날 어느 왕국에 공주님이 태어났어요. “어쩜 이렇게 어여쁠까? 살결이 눈처럼 하얗구나. 백 설공주라고 불러야겠다.” 왕과 왕비는 갓 태어난 딸을 보며 기뻐했어요. 하지만 기쁨도 잠시, 왕비는 곧 세상을 떠나고 말았어 요.'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': './data/snow-white.pdf', 'total_pages': 6, 'page': 1, 'page_label': '2'}, page_content='왕은 아름다운 새 왕비를 맞았어요. 그런데 새 왕비는 자기보다 아름다운 사람을 두고 보 지 못했어요. 왕비는 진실만을 말하는 요술 거울에게 늘 이렇게 물 었어요. “거울아, 거울아. 이 세상에서 누가 가장 아름답니?” “이 세상에서 가장 아름다운 사람은 왕비님입니다.” 그 대답을 들어야만 차가운 왕비 얼굴에 미소가 번졌 지요. 시간이 흘러 백설공주는 어여

### (2) Split

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,     # 각 chunk의 최대 문자 수 (기본값: 1000)
    chunk_overlap=20    # 인접한 텍스트 조각 간 겹치는 문자 수 (기본값: 200)
)                       # seperators: 텍스트 분할 구분자 우선순위 (기본값: ['\n\n', '\n', ' ', ''])

docs = splitter.split_documents(documents)
docs

[Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': './data/snow-white.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='백설공주 옛날 어느 왕국에 공주님이 태어났어요. “어쩜 이렇게 어여쁠까? 살결이 눈처럼 하얗구나. 백 설공주라고 불러야겠다.” 왕과 왕비는 갓 태어난 딸을 보며 기뻐했어요. 하지만'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': './data/snow-white.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='딸을 보며 기뻐했어요. 하지만 기쁨도 잠시, 왕비는 곧 세상을 떠나고 말았어 요.'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션',

### (3) Score

In [12]:
# 임베딩 모델 생성
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [10]:
#!pip install langchain-chroma

In [13]:
from langchain_chroma.vectorstores import Chroma

vector_store = Chroma.from_documents(docs, embedding_model)


- Retriever를 사용한 유사도 기반 검색

In [14]:
query = '백설공주와 왕비 중에 누가 더 아름답나요?'

# vector store 직접 조회
retrievals = (vector_store.similarity_search_with_score(query))
retrievals

[(Document(id='a889a376-2443-4db5-9c03-c61ee33650b3', metadata={'author': 'PC', 'creationdate': '2023-09-12T11:20:24+09:00', 'creator': 'Microsoft® PowerPoint® 2013', 'moddate': '2023-09-12T11:20:24+09:00', 'page': 1, 'page_label': '2', 'producer': 'Microsoft® PowerPoint® 2013', 'source': './data/snow-white.pdf', 'title': 'PowerPoint 프레젠테이션', 'total_pages': 6}, page_content='거울아. 이 세상에서 누가 가장 아름답니?” “왕비님도 아름답지만 백설공주가 더 아름답습니다.” 화가 난 왕비는 사냥꾼을 불렀어요. 왕비는 사냥꾼에게 백설공주를 죽이라고'),
  0.8399926424026489),
 (Document(id='684c276c-c606-4ab7-8f5b-08bb86fb5a61', metadata={'author': 'PC', 'creationdate': '2023-09-12T11:20:24+09:00', 'creator': 'Microsoft® PowerPoint® 2013', 'moddate': '2023-09-12T11:20:24+09:00', 'page': 2, 'page_label': '3', 'producer': 'Microsoft® PowerPoint® 2013', 'source': './data/snow-white.pdf', 'title': 'PowerPoint 프레젠테이션', 'total_pages': 6}, page_content='왕비는 다시 요술 거울에게 누가 가장 아름다운 지 물었어요. “왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.” “사냥꾼이 날 속였구나. 내가 직접 해치우겠어!”'),
  0.8604382276535034),
 

In [17]:
# Retriever를 사용한 검색
retriever = vector_store.as_retriever(
    search_type = 'similarity',
    search_kwargs={'k': 3}
)
retriever_result = retriever.batch([query])
retriever_result

[[Document(id='a889a376-2443-4db5-9c03-c61ee33650b3', metadata={'author': 'PC', 'creationdate': '2023-09-12T11:20:24+09:00', 'creator': 'Microsoft® PowerPoint® 2013', 'moddate': '2023-09-12T11:20:24+09:00', 'page': 1, 'page_label': '2', 'producer': 'Microsoft® PowerPoint® 2013', 'source': './data/snow-white.pdf', 'title': 'PowerPoint 프레젠테이션', 'total_pages': 6}, page_content='거울아. 이 세상에서 누가 가장 아름답니?” “왕비님도 아름답지만 백설공주가 더 아름답습니다.” 화가 난 왕비는 사냥꾼을 불렀어요. 왕비는 사냥꾼에게 백설공주를 죽이라고'),
  Document(id='684c276c-c606-4ab7-8f5b-08bb86fb5a61', metadata={'author': 'PC', 'creationdate': '2023-09-12T11:20:24+09:00', 'creator': 'Microsoft® PowerPoint® 2013', 'moddate': '2023-09-12T11:20:24+09:00', 'page': 2, 'page_label': '3', 'producer': 'Microsoft® PowerPoint® 2013', 'source': './data/snow-white.pdf', 'title': 'PowerPoint 프레젠테이션', 'total_pages': 6}, page_content='왕비는 다시 요술 거울에게 누가 가장 아름다운 지 물었어요. “왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.” “사냥꾼이 날 속였구나. 내가 직접 해치우겠어!”'),
  Document(id='36a158eb-c5c8-4b46-8a1d-c0c74b9d

# 2. Retrieval and Generation Phase

### (1) 프롬프트 생성

- 사용자 질의 + 검색된 문서

In [23]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ('system', '당신은 어린 아이에게 꿈과 희망을 심어주는 유치원 교사입니다. 질문하는 아이에게 최대한 호응하면서 context 기반으로만 답변해 주세요.'),
    ('user', '''
어린이의 질문에 context만을 이용해 답변하세요.
질문: {query}
context: {context}
''')
])

In [26]:
prompt.invoke({'query': query, 'context': retrievals})

ChatPromptValue(messages=[SystemMessage(content='당신은 어린 아이에게 꿈과 희망을 심어주는 유치원 교사입니다. 질문하는 아이에게 최대한 호응하면서 context 기반으로만 답변해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content="\n어린이의 질문에 context만을 이용해 답변하세요.\n질문: 백설공주가 마지막에 만난 사람은 누구인가요?\ncontext: [(Document(id='a889a376-2443-4db5-9c03-c61ee33650b3', metadata={'author': 'PC', 'creationdate': '2023-09-12T11:20:24+09:00', 'creator': 'Microsoft® PowerPoint® 2013', 'moddate': '2023-09-12T11:20:24+09:00', 'page': 1, 'page_label': '2', 'producer': 'Microsoft® PowerPoint® 2013', 'source': './data/snow-white.pdf', 'title': 'PowerPoint 프레젠테이션', 'total_pages': 6}, page_content='거울아. 이 세상에서 누가 가장 아름답니?” “왕비님도 아름답지만 백설공주가 더 아름답습니다.” 화가 난 왕비는 사냥꾼을 불렀어요. 왕비는 사냥꾼에게 백설공주를 죽이라고'), 0.8399926424026489), (Document(id='684c276c-c606-4ab7-8f5b-08bb86fb5a61', metadata={'author': 'PC', 'creationdate': '2023-09-12T11:20:24+09:00', 'creator': 'Microsoft® PowerPoint® 2013', 'moddate': '2023-09-12T11:20:24+09:00', 'page': 2, 'page_label': '3',

### (2) 모델 생성

In [27]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.5
)

### (3) Chain 생성

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# context 생성
query = "백설공주가 마지막으로 만난 사람은 누구인가요?"
retrievals = retriever.batch([query])
context_text = "\n".join([doc.page_content for doc in retrievals[0]])

chain = prompt | model | StrOutputParser()

### 사용

In [31]:
chain.invoke({'query': query, 'context': context_text})

'백설공주가 마지막으로 만난 사람은 누굴까요? 이야기를 보면 백설공주가 숲으로 도망쳤을 때, 그곳에서 여러 동물 친구들을 만났을 것 같아요. 그리고 나중에 그녀를 구해준 왕자님과도 만났겠죠! 그 왕자님이 백설공주를 정말 사랑하게 되었답니다. 꿈처럼 아름다운 이야기죠?'